# Story board

### 1. Data Exploration (yapping + some visualization about dataset) --> Haige
### 2. Initialize the baseline model + explain about how to improve it ("step 1" of instruction 3) --> Johnny
### 3. Initialize the model and explain about the development of each model one by one. (Here you also discuss the "step 2" of instruction 3) --> everyone
### 4. Compare the best version of everyone's model performance. --> Jason

## Section 1: Data Exploration

In [ ]:
# Implement here
...

## Section 2: Baseline Model

In [ ]:
# Implement here
...

## Sec 3 Model1: 

In [ ]:
# Implement here
...

## Sec 3 Model2: 

In [ ]:
# Implement here
...

# Sec 3 Model3: 

In [ ]:
# Implement here
...

## Sec 3 Model4:

In [ ]:
# Implement here
...

## Sec 3 Model5: 
#### Text Enbedding Model: TF-IDF
#### Data Analytics Model: Naive Bayes
#### Python Libaraies: Scikit-learn

In [ ]:
# Implement here
# Initialize the model and explain about the development of each model one by one. (Here you also discuss the "step 2" of instruction 3)

'''
1. Preparation of libraries, dataset paths, and sample texts.
'''
from model5.package.model import load_saved_model, evaluate_saved_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

pretrain_csv_path = 'Twitter_data/pre_traindata7.csv'
train_csv_path = 'Twitter_data/traindata7.csv'
test_csv_path = 'Twitter_data/testdata7.csv'

sample_texts = ["I love this!", "I hate this!", "Sounds alright."]

In [ ]:
'''
2. Test with standard model trained with standard dataset.
'''
# Initialize the models
model5_standard = load_saved_model('model5/package/saved_models')

model5_standard_tfidf_model: TfidfVectorizer = model5_standard['embedding']
model5_standard_nb_model: MultinomialNB = model5_standard['model']

# Evaluate the model on training and testing datasets
model5_standard_eval_result = evaluate_saved_model(model5_standard, train_csv_path, test_csv_path)

model5_standard_train_loss      = model5_standard_eval_result.get('train').get('loss')
model5_standard_train_accuracy  = model5_standard_eval_result.get('train').get('accuracy')
model5_standard_test_loss       = model5_standard_eval_result.get('test').get('loss')
model5_standard_test_accuracy   = model5_standard_eval_result.get('test').get('accuracy')

print("Evaluation Results for Standard Model:")
print(f"Train loss: {model5_standard_train_loss}, Train accuracy: {model5_standard_train_accuracy}")
print(f"Test loss: {model5_standard_test_loss}, Test accuracy: {model5_standard_test_accuracy}")

# Prediction Result
embedded_texts = model5_standard_tfidf_model.transform(sample_texts)
nb_predictions = model5_standard_nb_model.predict(embedded_texts)

print("Sample Predictions:")
for t, p in zip(sample_texts, nb_predictions):
    print(f"Text: '{t}' => Predicted Sentiment: {p}")

In [ ]:
'''
3. Test with standard model trained with pre-trained dataset.
'''
# Initialize the models
model5_pretrain = load_saved_model('model5/package/saved_models_pretrain7')

model5_pretrain_tfidf_model: TfidfVectorizer = model5_pretrain['embedding']
model5_pretrain_nb_model: MultinomialNB = model5_pretrain['model']

# Evaluate the model on training and testing datasets
model5_pretrain_eval_result = evaluate_saved_model(model5_pretrain, pretrain_csv_path, test_csv_path)

model5_pretrain_train_loss      = model5_pretrain_eval_result.get('train').get('loss')
model5_pretrain_train_accuracy  = model5_pretrain_eval_result.get('train').get('accuracy')
model5_pretrain_test_loss       = model5_pretrain_eval_result.get('test').get('loss')
model5_pretrain_test_accuracy   = model5_pretrain_eval_result.get('test').get('accuracy')

print("Evaluation Results for Pretrained Model:")
print(f"Train loss: {model5_pretrain_train_loss}, Train accuracy: {model5_pretrain_train_accuracy}")
print(f"Test loss: {model5_pretrain_test_loss}, Test accuracy: {model5_pretrain_test_accuracy}")

# Prediction Result
embedded_texts = model5_pretrain_tfidf_model.transform(sample_texts)
nb_predictions = model5_pretrain_nb_model.predict(embedded_texts)

print("Sample Predictions:")
for t, p in zip(sample_texts, nb_predictions):
    print(f"Text: '{t}' => Predicted Sentiment: {p}")

#### Explanation later

## Sec 3 Model6:

In [ ]:
# Implement here
...

## Section 4: Comparison

In [ ]:
# Implement here
...